# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
%matplotlib inline
import matplotlib.pyplot as plt
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords','averaged_perceptron_tagger'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)
X = df['message'].values
y = df.iloc[:,4:].values

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    # remove URLs
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # normalize
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]",' ',text)
    
    #tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    
    # Lemmatize
    tokens = [WordNetLemmatizer().lemmatize(i) for i in tokens]
    return tokens

In [4]:
text = X[0]
text, tokenize(text)

('Weather update - a cold front from Cuba that could pass over Haiti',
 ['weather',
  'update',
  'a',
  'cold',
  'front',
  'from',
  'cuba',
  'that',
  'could',
  'pas',
  'over',
  'haiti'])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def display_results(y_test, y_pred):
    for i in range(y_test.shape[1]): 
        print('*******',df.columns[4:][i],'*******')
        print(classification_report(y_test[:,1], y_pred[:,1]))

In [9]:
y_train_pred = pipeline.predict(X_train)

In [12]:
display_results(y_train, y_train_pred)

******* related *******
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.94      0.97      3363

avg / total       0.99      0.99      0.99     19662

******* request *******
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.94      0.97      3363

avg / total       0.99      0.99      0.99     19662

******* offer *******
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.94      0.97      3363

avg / total       0.99      0.99      0.99     19662

******* aid_related *******
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.94      0.97      3363

avg / total       0.99      0.99      0.99     19662

******* medical_help *******
             

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {
        'clf__estimator__n_estimators': [20, 50, 100]
    }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3, verbose=3)

In [18]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__n_estimators=20 .................................
[CV]  clf__estimator__n_estimators=20, score=0.21513579493439122, total= 1.1min
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


[CV]  clf__estimator__n_estimators=20, score=0.21696673787000306, total= 1.1min
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=20, score=0.22230698809887092, total= 1.1min
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.22886786695148, total= 2.6min
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.21833994507171192, total= 2.6min
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.22017088800732376, total= 2.7min
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=100, score=0.21742447360390602, total= 5.1min
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=100, score=0.21650900213610008, total= 5.1min
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=100, score=0.22444308819041806, total= 5.2min


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 29.1min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [20, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
y_test_pred = cv.predict(X_test)

In [22]:
display_results(y_test, y_test_pred)

******* related *******
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.94      0.97      3363

avg / total       0.99      0.99      0.99     19662

******* request *******
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.94      0.97      3363

avg / total       0.99      0.99      0.99     19662

******* offer *******
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.94      0.97      3363

avg / total       0.99      0.99      0.99     19662

******* aid_related *******
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.94      0.97      3363

avg / total       0.99      0.99      0.99     19662

******* medical_help *******
             

******* other_weather *******
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.94      0.97      3363

avg / total       0.99      0.99      0.99     19662

******* direct_report *******
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.94      0.97      3363

avg / total       0.99      0.99      0.99     19662



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [5]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        words = tokenize(text)
        pos_tags = nltk.pos_tag(words)
        if pos_tags !=[]:
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [6]:
pipeline2 = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
])

In [7]:
sv = StartingVerbExtractor()
sv.starting_verb(X[0])

False

In [10]:
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))])

In [14]:
parameters2 = {
     'features__text_pipeline__vect__max_df': (0.5,  1.0),
#     'features__text_pipeline__tfidf__use_idf': (True, False),
     'clf__estimator__n_estimators': [20, 50],
#      'features__transformer_weights': (
#         {'text_pipeline': 1, 'starting_verb': 0.5},
#         {'text_pipeline': 0.5, 'starting_verb': 1},
#     )
}

In [15]:
model = GridSearchCV(estimator=pipeline2, param_grid=parameters2,cv=3, verbose=3)

In [16]:
model.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=0.5 
[CV]  clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=0.5, score=0.22078120231919438, total= 1.8min
[CV] clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=0.5, score=0.2145254806225206, total= 1.8min
[CV] clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=0.5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.9min remaining:    0.0s


[CV]  clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=0.5, score=0.21849252364967958, total= 1.8min
[CV] clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=1.0 
[CV]  clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=1.0, score=0.22322245956667683, total= 1.8min
[CV] clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=1.0 
[CV]  clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=1.0, score=0.21269453768690877, total= 1.8min
[CV] clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=1.0 
[CV]  clf__estimator__n_estimators=20, features__text_pipeline__vect__max_df=1.0, score=0.21437290204455295, total= 1.8min
[CV] clf__estimator__n_estimators=50, features__text_pipeline__vect__max_df=0.5 
[CV]  clf__estimator__n_estimators=50, features__text_pipeline__vect__max_df=0.5, score=0.22215440952090326, total= 3.3min
[CV] clf__estimator__n_estimators=50, features__text_pipeline

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 38.7min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__vect__max_df': (0.5, 1.0), 'clf__estimator__n_estimators': [20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [17]:
y_test_pred = model.predict(X_test)

In [22]:
display_results(y_test, y_test_pred)

******* related *******
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      5440
          1       0.88      0.46      0.60      1114

avg / total       0.90      0.90      0.88      6554

******* request *******
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      5440
          1       0.88      0.46      0.60      1114

avg / total       0.90      0.90      0.88      6554

******* offer *******
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      5440
          1       0.88      0.46      0.60      1114

avg / total       0.90      0.90      0.88      6554

******* aid_related *******
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      5440
          1       0.88      0.46      0.60      1114

avg / total       0.90      0.90      0.88      6554

******* medical_help *******
             

In [25]:
model.best_params_

{'clf__estimator__n_estimators': 50,
 'features__text_pipeline__vect__max_df': 0.5}

### 9. Export your model as a pickle file

In [28]:
with open('CV_model.pkl', 'wb') as file:
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.